# 9 - Metaprogramming

## Putting a Wrapper Around a Function

In [1]:
import time
from functools import wraps

def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper


In [2]:
@timethis
def countdown(n):
    while n > 0:
        n -= 1


In [5]:
countdown(1e6)

countdown 0.07303404808044434


## Preserving Function Metadata When Writing Decorators
Remember to use @wraps...

In [8]:
def action(func):
    def wrapper(*args, **kwargs):
        ''' Docs for wrapper '''
        result = func(*args, **kwargs)
        return result
    return wrapper

@action
def adder(a, b):
    ''' Docs for adder '''
    return a + b

adder(1, 2)

3

In [11]:
adder.__name__

'wrapper'

In [13]:
adder.__doc__

' Docs for wrapper '

In [14]:
adder.__annotations__

{}

If we use @wraps instead...

In [16]:
from functools import wraps

def action(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        ''' Docs for wrapper '''
        result = func(*args, **kwargs)
        return result
    return wrapper

@action
def adder(a, b):
    ''' Docs for adder '''
    return a + b

adder(1, 2)

3

In [17]:
adder.__name__

'adder'

In [18]:
adder.__doc__

' Docs for adder '

In [19]:
adder.__annotations__

{}